In [1]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

In [2]:
z = torch.FloatTensor([1, 2, 3])
hypothesis = F.softmax(z, dim = 0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [4]:
print(hypothesis.sum())

tensor(1.)


In [5]:
z = torch.rand(3, 5, requires_grad = True)
print(z)

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)


In [6]:
hypothesis = F.softmax(z, dim = 1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [7]:
print(hypothesis.sum())

tensor(3.0000, grad_fn=<SumBackward0>)


In [8]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [9]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [10]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim = 1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [12]:
print(torch.log(F.softmax(z, dim = 1)))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)


In [13]:
print(F.log_softmax(z, dim = 1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)


In [14]:
F.nll_loss(F.log_softmax(z, dim = 1), y) # nll: Negative Log Likelihood

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [15]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [17]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [18]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [19]:
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot)

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


In [22]:
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros((1, 3), requires_grad = True)
optimizer = optim.SGD([W, b], lr=0.01)

torch.Size([3, 5])
torch.Size([3, 5])


In [24]:
epochs = 1000
for epoch in range(epochs):
    hypothesis = F.softmax(x_train.matmul(W) + b, dim = 1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim = 1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.540759
Epoch  100/1000 Cost: 0.529122
Epoch  200/1000 Cost: 0.518616
Epoch  300/1000 Cost: 0.509045
Epoch  400/1000 Cost: 0.500253
Epoch  500/1000 Cost: 0.492121
Epoch  600/1000 Cost: 0.484553
Epoch  700/1000 Cost: 0.477470
Epoch  800/1000 Cost: 0.470809
Epoch  900/1000 Cost: 0.464518


In [26]:
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros((1, 3), requires_grad = True)
optimizer = optim.SGD([W, b], lr=0.01)

epochs = 1000
for epoch in range(epochs + 1):
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
        ))


Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.825978
Epoch  200/1000 Cost: 0.745367
Epoch  300/1000 Cost: 0.695094
Epoch  400/1000 Cost: 0.658135
Epoch  500/1000 Cost: 0.629088
Epoch  600/1000 Cost: 0.605386
Epoch  700/1000 Cost: 0.585531
Epoch  800/1000 Cost: 0.568553
Epoch  900/1000 Cost: 0.553787
Epoch 1000/1000 Cost: 0.540759


In [28]:
model = nn.Linear(4, 3)
optimizer = optim.SGD(model.parameters(), lr=0.1)
epochs = 1000
for epoch in range(epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
        ))


Epoch    0/1000 Cost: 1.616785
Epoch  100/1000 Cost: 0.658891
Epoch  200/1000 Cost: 0.573443
Epoch  300/1000 Cost: 0.518151
Epoch  400/1000 Cost: 0.473265
Epoch  500/1000 Cost: 0.433516
Epoch  600/1000 Cost: 0.396563
Epoch  700/1000 Cost: 0.360914
Epoch  800/1000 Cost: 0.325392
Epoch  900/1000 Cost: 0.289178
Epoch 1000/1000 Cost: 0.254148


In [33]:
class SoftMaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)
    def forward(self, x):
        return self.linear(x)

In [34]:
model = SoftMaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [35]:
epochs = 1000
for epoch in range(epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
        ))


Epoch    0/1000 Cost: 2.637636
Epoch  100/1000 Cost: 0.647903
Epoch  200/1000 Cost: 0.564643
Epoch  300/1000 Cost: 0.511043
Epoch  400/1000 Cost: 0.467249
Epoch  500/1000 Cost: 0.428281
Epoch  600/1000 Cost: 0.391924
Epoch  700/1000 Cost: 0.356742
Epoch  800/1000 Cost: 0.321577
Epoch  900/1000 Cost: 0.285617
Epoch 1000/1000 Cost: 0.250818
